In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
import h5py
import os
import cv2
import csv
import math
import tensorflow as tf
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

Using TensorFlow backend.


In [2]:
train_classifier_dir = 'C:/Users/abhis/Pictures/Smoke_Dataset/Train'
valid_classifier_dir = 'C:/Users/abhis/Pictures/Smoke_Dataset/Valid'

img_width, img_height = 224,224

datagen = ImageDataGenerator(rescale=1./255)

train_classifier = datagen.flow_from_directory(
        train_classifier_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary',
        shuffle='False')

valid_classifier = datagen.flow_from_directory(
        valid_classifier_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='binary',
        shuffle='False')

print('complete')

Found 672 images belonging to 2 classes.
Found 79 images belonging to 2 classes.
complete


In [4]:
#del classifier
classifier = Sequential()

classifier.add(Conv2D(64,(5,5),input_shape=(224,224,3)))
classifier.add(Conv2D(128,(3,3)))
classifier.add(MaxPooling2D())
classifier.add(Activation('relu'))
classifier.add(Conv2D(256,(2,2)))
classifier.add(Conv2D(256,(2,2)))
classifier.add(MaxPooling2D())
classifier.add(Activation('relu'))
classifier.add(Conv2D(512,(2,2)))
classifier.add(Conv2D(512,(2,2)))
classifier.add(MaxPooling2D())
classifier.add(Activation('relu'))
classifier.add(Flatten())
classifier.add(Dense(50,activation='relu'))
classifier.add(Dense(1,activation='sigmoid'))

In [5]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 220, 220, 64)      4864      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 218, 218, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 109, 128)     0         
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 256)     131328    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 107, 107, 256)     262400    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 256)       0         
__________

In [6]:
classifier.load_weights('Z:/Smoke_Classifier_weights/StackedModelClassifier0.044.h5')
print('Loading Complete')

Loading Complete


In [7]:
tf.global_variables_initializer()
tf.local_variables_initializer()

classifier.compile(optimizer=keras.optimizers.SGD(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
classifier.evaluate_generator(valid_classifier,steps=1,verbose=1)
#classifier.evaluate_generator(train_classifier,steps=37,verbose=1)

1/1 [==============================] - 16s 16s/step


[0.4198302626609802, 0.800000011920929]

In [8]:
"""
history_classifier = classifier.fit_generator(train_classifier,
                                              steps_per_epoch=42,
                                              epochs=50,
                                              validation_data=valid_classifier,
                                              validation_steps=8)
"""

'\nhistory_classifier = classifier.fit_generator(train_classifier,\n                                              steps_per_epoch=42,\n                                              epochs=50,\n                                              validation_data=valid_classifier,\n                                              validation_steps=8)\n'

In [29]:
# Save tf.keras model in HDF5 format.
#tf.global_variables_initializer()
keras_file = "keras_smoke_model.h5"
tf.keras.models.save_model(classifier, keras_file)

# Convert to TensorFlow Lite model.
#converter = tf.contrib.lite.toco_convert.from_keras_model_file(keras_file)
#tflite_model = converter.convert()
#open("converted_model.tflite", "wb").write(tflite_model)


FailedPreconditionError: Attempting to use uninitialized value conv2d_13/bias
	 [[Node: conv2d_13/bias/_32 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6_conv2d_13/bias", _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_13/bias)]]
	 [[Node: conv2d_13/kernel/_35 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8_conv2d_13/kernel", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
print(history_classifier.history.keys())
#  "Accuracy"
plt.plot(history_classifier.history['acc'])
plt.plot(history_classifier.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history_classifier.history['loss'])
plt.plot(history_classifier.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [14]:
from keras.utils import Sequence

train_images_path = 'Z:/Smoke_DataSet2/Train/'
valid_images_path = 'Z:/Smoke_DataSet2/Valid/'
train_CSV_file = 'Z:/Smoke_DataSet2/Train.csv'
valid_CSV_file = 'Z:/Smoke_DataSet2/Valid.csv'
img_size = 224

class DataGenerator(Sequence):
    
    def __init__(self,csv_file,img_path,num,batch_size=25):
        
        self.csv_file = csv_file
        self.x = []
        self.y = np.zeros((int(num),4))
        self.batch_size = batch_size
        self.length = 0
        
        with open(self.csv_file,"r") as csv_data:
            csv_reader = csv.reader(csv_data, delimiter=',')
            line_no = 0
            
            for row in csv_reader:
                if line_no != 0:
                    
                    w = int(row[1])
                    h = int(row[2])
                    minx = abs(int(row[4]))
                    miny = abs(int(row[5]))
                    maxx = abs(int(row[6]))
                    maxy = abs(int(row[7]))
                    
                    self.y[line_no-1][0] = (minx/w)*img_size
                    self.y[line_no-1][1] = (miny/h)*img_size
                    self.y[line_no-1][2] = (maxx/w)*img_size
                    self.y[line_no-1][3] = (maxy/h)*img_size
                    
                    #self.y.append([minx,miny,maxx,maxy])
                    self.x.append(img_path + row[0])
                    line_no += 1
                    
                else:
                    line_no += 1
        self.length = (math.ceil(len(self.x) / self.batch_size))
        #print(len(self.x),self.length)
        #print(self.y)
                    
    def __loadimages(self,batch):
            
        img = []
            
        for filename in batch:
                
            im = cv2.resize(cv2.imread(filename), (img_size, img_size))
            img.append(im)
                
        return np.array(img)
            
    def __len__(self):
        return self.length
                
    def __getitem__(self,idx):
            
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
            
        images = self.__loadimages(batch_x).astype('float32')
            
        return images,batch_y
            

In [15]:
train_datagen = DataGenerator(train_CSV_file,train_images_path,955,20)
valid_datagen = DataGenerator(valid_CSV_file,valid_images_path,100,10)

In [16]:
#del Localizer
Localizer = Sequential()

Localizer.add(Conv2D(64,(5,5),input_shape=(224,224,3)))
Localizer.add(Conv2D(128,(3,3)))
Localizer.add(MaxPooling2D())
Localizer.add(Activation('relu'))
Localizer.add(Conv2D(256,(2,2)))
Localizer.add(Conv2D(256,(2,2)))
Localizer.add(MaxPooling2D())
Localizer.add(Activation('relu'))
Localizer.add(Conv2D(512,(2,2)))
Localizer.add(Conv2D(512,(2,2)))
Localizer.add(MaxPooling2D())
Localizer.add(Activation('relu'))
Localizer.add(Flatten())
Localizer.add(Dense(50,activation='relu'))
Localizer.add(Dense(4,activation='relu'))

In [17]:
Localizer.compile(loss=keras.losses.mean_squared_logarithmic_error, 
              optimizer=keras.optimizers.SGD(lr=0.01), 
              metrics=["accuracy"])

In [ ]:
Localizer.fit_generator(train_datagen, 
                    steps_per_epoch=48, 
                    epochs=5, 
                    validation_data=valid_datagen,
                    validation_steps=10)

Epoch 1/5
48/48 [==============================] - 449s 9s/step - loss: 23.8332 - acc: 0.0833 - val_loss: 17.2581 - val_acc: 0.0000e+00
Epoch 2/5
48/48 [==============================] - 385s 8s/step - loss: 19.1876 - acc: 0.0000e+00 - val_loss: 17.2581 - val_acc: 0.0000e+00
Epoch 3/5
37/48 [======================>.......] - ETA: 1:30 - loss: 19.1076 - acc: 0.0000e+00